# we can run this on Colab!

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Date Precossing part II

## Create this trans_real function to turn all the cells from string to list

In [15]:
def trans_real(x):
    try:
        return eval(x)[0]
    except:
        pass

In [17]:
demon = pd.read_csv('Laplace\'s demon.csv',low_memory=False)

demon_real = pd.DataFrame()

for col in demon:
    if col != 'time':
        demon_real[col] = demon[col].apply(trans_real)
    
demon_real.drop('Unnamed: 0', axis=1, inplace=True)

### Let's look at the result!

In [20]:
demon_real['tdc6p19'][0]

{'type': 'P',
 'service_category': [{'category': 'HOURLY',
   'vacancy_type': 'A',
   'vacancy': 0,
   'lastupdate': '2020-06-05 14:17:18'}]}

## Create this get_types function to get the type and vacancy type from all the cells

In [21]:
def get_types(x):
    try:
        return (x['type'], x['service_category'][0]['vacancy_type'])
    except:
        pass

In [22]:
types = pd.DataFrame()

for col in demon_real:
    if col != 'time':
        types[col] = demon_real[col].apply(get_types)

### Let's have a look at the types

In [26]:
types.head()

,tdc6p10,tdc6p11,tdc6p12,tdc6p8,tdc6p9,tdcp1,tdcp10,tdcp11,tdcp12,tdcp2,...,tdc52p1,tdc62p1,tdc72p1,tdc74p1,tdc74p2,tdc74p3,tdc74p4,tdc74p5,tdc49p1,tdc49p2
0,"(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, A)","(P, A)","(P, A)","(P, A)",...,None,None,None,None,None,None,None,None,None,None
1,"(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, A)","(P, A)","(P, A)","(P, A)",...,None,None,None,None,None,None,None,None,None,None
2,"(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, A)","(P, A)","(P, A)","(P, A)",...,None,None,None,None,None,None,None,None,None,None
3,"(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, A)","(P, A)","(P, A)","(P, A)",...,None,None,None,None,None,None,None,None,None,None
4,"(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, B)","(P, A)","(P, A)","(P, A)","(P, A)",...,None,None,None,None,None,None,None,None,None,None


## Filter out unwanted types

When Vacancy Type is A:
> 0 means car park is full

> -1 means "No data provided by the car park
operator

When Vacancy Type is B:
> 0 means car park is full
 
> 1 means parking space available

> -1 means "No data provided by the car park
operator

When Vacancy Type is C:
> it always returns 0 (the car park is closed)

### We want types (P, A) and (P, B)

We only look at private cars which are type P. 

We don't want vacancy type C because it is meaningless to predict the vacancy if the car park is shut down or unstably operating.

In [31]:
qualified_cols = []

for col, colData in types.iteritems():
    ls = list(types[col].unique())
    if ('P', 'A') in ls and ('P', 'C') not in ls:
        qualified_cols.append(col)

In [30]:
len(qualified_cols)

150

## Retrieve vacanies

In [32]:
def get_vac(x):
    try:
        return (x['service_category'][0]['vacancy'])
    except:
        pass

In [36]:
vacanies = pd.DataFrame()

for col in qualified_cols:
    vacanies[col] = demon_real[col].apply(get_vac)
    
vacanies['time'] = demon['time']
vacanies['time'] = pd.to_datetime(vacanies['time'])

vacanies['day_of_week'] = vacanies['time'].apply(lambda x: x.dayofweek)
vacanies['minute'] = vacanies['time'].apply(lambda x: 15*(x.minute // 15))
vacanies['hour'] = vacanies['time'].apply(lambda x: x.hour)

### Let's take a look!

In [43]:
vacanies

,tdc6p12,tdcp10,tdcp11,tdcp12,tdcp2,tdcp3,tdcp5,tdcp6,tdcp7,tdcp8,...,tdc74p1,tdc74p2,tdc74p3,tdc74p4,tdc49p1,tdc49p2,time,day_of_week,minute,hour
0,NaN,30.0,155.0,61.0,489.0,136,131,164,68.0,293,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-10 00:00:00,2,0,0
1,NaN,30.0,156.0,61.0,491.0,136,131,163,68.0,288,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-10 00:15:00,2,15,0
2,NaN,30.0,156.0,61.0,493.0,136,131,163,68.0,290,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-10 00:30:00,2,30,0
3,NaN,30.0,155.0,61.0,495.0,136,131,163,68.0,290,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-10 00:45:00,2,45,0
4,NaN,30.0,156.0,61.0,496.0,136,131,162,69.0,290,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-10 01:00:00,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9498,NaN,23.0,153.0,60.0,504.0,112,131,146,46.0,293,...,0.0,0.0,0.0,0.0,489.0,503.0,2021-06-16 22:30:00,2,30,22
9499,NaN,27.0,154.0,60.0,505.0,112,131,149,66.0,295,...,0.0,0.0,0.0,0.0,489.0,503.0,2021-06-16 22:45:00,2,45,22
9500,NaN,29.0,154.0,60.0,505.0,113,133,148,67.0,297,...,0.0,0.0,0.0,0.0,489.0,503.0,2021-06-16 23:00:00,2,0,23
9501,NaN,29.0,154.0,61.0,508.0,113,133,148,67.0,297,...,0.0,0.0,0.0,0.0,490.0,504.0,2021-06-16 23:15:00,2,15,23


## Fill in NaN

> vacany type A and B have -1 as vacany which means NaN

> vacany type B with vacany 1 means the car park has space, but not known, we change these to NaN also

In [39]:
sum(vacanies.isnull().sum())

254564

In [40]:
for col in vacanies:
    if col == 'time':
        break
    for i, val in enumerate(vacanies[col]):
        if val == -1:
            vacanies[col][i] = float('NaN')
        if val == 1 and types[col][i] == ('P', 'B'):
            vacanies[col][i] = float('NaN')

get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get


<ipython-input-40-02d4ac218266>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][i] = float('NaN')
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)



get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get

get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get


get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get


...

get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get


get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get


get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get


get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get


get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get
get


In [42]:
sum(vacanies.isnull().sum())

320253

### See which car parks have null value

We will remove car parks which missing values more than 10%

In [50]:
nulls = (vacanies.isnull().sum()/9503 * 100).to_frame()

In [51]:
to_remove = list(null_df[nulls[0] > 10].index)

Number of car parks need to be removed

In [52]:
len(to_remove)

56

### Save the qualified 150 car parks with vacanies as qualified_vacany.csv

In [55]:
vacanies.to_csv('qualified_vacany.csv')

### Remove car parks with null rate < 10% and save it as qualified_vacany_pure.csv

In [56]:
for col in to_remove:
    vacanies.drop(col, axis=1, inplace=True)

vacanies.to_csv('qualified_vacany_pure.csv')

### Fill in NaN

we fill in NaN by the mean of vacanies at a particular 'day_of_week', 'hour', 'minute'

In [64]:
count_df = vacanies.isnull().sum().to_frame()

to_fill = list(count_ls[count_df[0] > 0].index)

we group the car parks by their 'day_of_week', 'hour', 'minute' and then take mean. As a result, we know the mean of the vacanies of all car parks at a particular 'day_of_week', 'hour', 'minute'.

In [68]:
grouped_times = vacanies.groupby(['day_of_week', 'hour', 'minute']).mean()[to_fill]

let's take a look!

In [69]:
grouped_times

tdcp10      tdcp11     tdcp12       tdcp2  \
day_of_week hour minute                                                 
0           0    0       28.785714  154.384615  63.857143  508.214286   
                 15      28.857143  154.692308  63.857143  508.785714   
                 30      26.642857  155.000000  63.785714  509.285714   
                 45      28.785714  155.230769  63.785714  471.785714   
            1    0       28.857143  155.307692  63.857143  472.071429   
...                            ...         ...        ...         ...   
6           22   45      28.214286  151.307692  62.785714  503.214286   
            23   0       28.785714  152.384615  62.857143  505.357143   
                 15      28.642857  152.923077  63.214286  506.428571   
                 30      28.714286  153.076923  63.500000  507.214286   
                 45      28.714286  153.846154  63.571429  507.642857   

                             tdcp7      tdcp9   tdstt2p2  tdc17p1      tdc1p5  \
day_of_week hour minute                                                         
0           0    0       65.785714  92.928571  19.071429     49.0  144.642857   
                 15      66.071429  93.142857  19.142857     49.0  144.642857   
                 30      66.000000  93.500000  19.142857     49.0  144.642857   
                 45      66.214286  93.642857  19.142857     49.0  144.571429   
            1    0       66.285714  93.857143  19.142857     49.0  144.642857   
...                            ...        ...        ...      ...         ...   
6           22   45      62.500000  91.571429  18.714286     49.0  143.142857   
            23   0       63.785714  92.214286  18.785714     49.0  143.642857   
                 15      64.785714  92.285714  19.000000     49.0  143.785714   
                 30      65.428571  92.142857  18.928571     49.0  144.142857   
                 45      65.571429  92.642857  19.071429     49.0  144.285714   

                            tdc1p4  ...    tdc38p2   tdc42p1    tdc41p1  \
day_of_week hour minute             ...                                   
0           0    0       47.571429  ...  39.714286  0.928571  18.153846   
                 15      47.357143  ...  39.714286  1.000000  17.916667   
                 30      47.571429  ...  39.714286  1.000000  17.923077   
                 45      47.928571  ...  39.714286  1.000000  17.923077   
            1    0       48.071429  ...  39.714286  1.000000  17.846154   
...                            ...  ...        ...       ...        ...   
6           22   45      46.214286  ...  36.928571  0.500000  18.692308   
            23   0       46.500000  ...  38.000000  0.785714  18.846154   
                 15      46.928571  ...  39.142857  0.571429  18.384615   
                 30      47.071429  ...  39.714286  0.928571  18.461538   
                 45      47.000000  ...  39.714286  0.857143  18.166667   

                         tdc41p2  tdstt39  tdstt40  tdstt41  tdstt42  \
day_of_week hour minute                                                
0           0    0           9.0     30.0     30.0     30.0     30.0   
                 15          9.0     30.0     30.0     30.0     30.0   
                 30          9.0     30.0     30.0     30.0     30.0   
                 45          9.0     30.0     30.0     30.0     30.0   
            1    0           9.0     30.0     30.0     30.0     30.0   
...                          ...      ...      ...      ...      ...   
6           22   45          9.0     30.0     30.0     30.0     30.0   
            23   0           9.0     30.0     30.0     30.0     30.0   
                 15          9.0     30.0     30.0     30.0     30.0   
                 30          9.0     30.0     30.0     30.0     30.0   
                 45          9.0     30.0     30.0     30.0     30.0   

                           tdstt35   tdc25p43  
day_of_week hour minute                        
0           0  

Find the location of the missing value

In [70]:
def find_time_index(ready_df, to_fill):
    timess = []
    indexs = []
    
    for i, col in enumerate(to_fill):
        index = list(ready_df[[to_fill[i], 'day_of_week', 'hour', 'minute']][ready_df[to_fill[i]].isnull()][['day_of_week', 'hour', 'minute']].index)
        times = ready_df[[to_fill[i], 'day_of_week', 'hour', 'minute']][ready_df[to_fill[i]].isnull()][['day_of_week', 'hour', 'minute']].values
        timess.append(times)
        indexs.append(index)
    
    return timess, indexs

In [72]:
timess, indexs = find_time_index(vacanies, to_fill)

In [74]:
for i, value in enumerate(timess):
    col = to_fill[i]
    for j, time in enumerate(value):
        day = time[0]
        hour = time[1]
        minute = time[2]
        fill_mean = grouped_times.query('day_of_week == ' + str(day) + ' and hour == ' + str(hour) + ' and minute == ' + str(minute))[col].values[0]
        vacanies[col][indexs[i][j]] = fill_mean

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

...

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacanies[col][indexs[i][j]] = fill_mean
<ipython-input-74-55ee9edaf46c>:8: SettingWithCopyWarning: 
A value is tryin

In [76]:
vacanies.to_csv('qualified_vacany_pure_filled.csv')